In [8]:
import torch
from torch import nn
from torch.utils.data import DataLoader, TensorDataset, random_split
from torchvision import datasets
from torchvision.transforms import ToTensor
import pandas as pd

In [9]:
# Loading datasets

training_labels = pd.read_csv('/home/sesso/Documents/Study/pytorch_learn/volcanoes_venus/venus_data/archive/volcanoes_train/train_labels.csv')
training_input = pd.read_csv('/home/sesso/Documents/Study/pytorch_learn/volcanoes_venus/venus_data/archive/volcanoes_train/train_images.csv',
                                header = None)

testing_labels = pd.read_csv('/home/sesso/Documents/Study/pytorch_learn/volcanoes_venus/venus_data/archive/volcanoes_test/test_labels.csv')
testing_input = pd.read_csv('/home/sesso/Documents/Study/pytorch_learn/volcanoes_venus/venus_data/archive/volcanoes_test/test_images.csv',
                                header = None)           

# Creating tensor dataset

target_col = 'Type'
# use 30% of training data for validation
train_set_size = int(len(training_input) * 0.7)
valid_set_size = len(training_input) - train_set_size
batch_size = 64

## Convert to numpy array

training_labels = training_labels[target_col].to_numpy()
training_input = training_input.to_numpy()

testing_labels = testing_labels[target_col].to_numpy()
testing_input = testing_input.to_numpy()

## Make tensors of them

training_inputs = torch.tensor(training_input, dtype=torch.float32)
training_targets = torch.tensor(training_labels, dtype=torch.float32)

testing_inputs = torch.tensor(testing_input, dtype=torch.float32)
testing_targets = torch.tensor(testing_labels, dtype=torch.float32)

## Tensor datasets

dtrain = TensorDataset(training_inputs, training_targets)
dtest = TensorDataset(testing_inputs, testing_targets)

## Split train and validation set

### split the train set into two
seed = torch.Generator().manual_seed(42)
train_set, valid_set = random_split(dtrain, [train_set_size, valid_set_size], generator=seed)

### Batch loader

train_dl = DataLoader(train_set, batch_size = batch_size, shuffle=True) 
val_dl = DataLoader(valid_set, batch_size = batch_size, shuffle=True) 
test_dl = DataLoader(dtest, batch_size =batch_size)



In [ ]:
batch_size = 64

# Create data loaders.
train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)

for X, y in test_dataloader:
    print(f"Shape of X [N, C, H, W]: {X.shape}")
    print(f"Shape of y: {y.shape} {y.dtype}")
    break